In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-lens-small-latest-dataset/movies.csv
/kaggle/input/movie-lens-small-latest-dataset/ratings.csv
/kaggle/input/movie-lens-small-latest-dataset/README.txt
/kaggle/input/movie-lens-small-latest-dataset/tags.csv
/kaggle/input/movie-lens-small-latest-dataset/links.csv


# Step 1 : Read movies and rating file and select required columns

In [2]:
movies = pd.read_csv ("/kaggle/input/movie-lens-small-latest-dataset/movies.csv", usecols=['movieId','title'],dtype={'movieId':'int64','title':'str'})
rating = pd.read_csv("/kaggle/input/movie-lens-small-latest-dataset/ratings.csv", usecols=['userId','movieId','rating'],dtype={'userId':'int64','movieId':'int64','rating':'float64'})


In [3]:
# After selecting columns check head of movies
movies.head(3)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)


In [4]:
# create ne coloumn name as year from title
movies["Year"] = movies.title.str.extract("(\(\d\d\d\d\))" ,expand = True)
movies["Year"]

0       (1995)
1       (1995)
2       (1995)
3       (1995)
4       (1995)
         ...  
9737    (2017)
9738    (2017)
9739    (2017)
9740    (2018)
9741    (1991)
Name: Year, Length: 9742, dtype: object

In [5]:
# Removing extra brakets 
movies["Year"] = movies.title.str.extract("(\d\d\d\d)" ,expand = True)
movies["Year"]

0       1995
1       1995
2       1995
3       1995
4       1995
        ... 
9737    2017
9738    2017
9739    2017
9740    2018
9741    1991
Name: Year, Length: 9742, dtype: object

In [6]:
# Replace year and add whitespaces
movies["title"]= movies.title.str.replace("(\(\d\d\d\d\))","")

# Check movies data
movies

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,movieId,title,Year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017
9738,193583,No Game No Life: Zero,2017
9739,193585,Flint,2017
9740,193587,Bungo Stray Dogs: Dead Apple,2018


In [7]:
# After droping colums check head of ratings
rating.head(3)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0


# Step 2 : Create Pivot table

In [8]:
# Create pivot table and set index as movieId , column as userId and values as rating
movies_users=rating.pivot_table(index='movieId', columns='userId',values='rating').fillna(0)

# Check head of movies_user
movies_users.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


# Step 3 : Import csr_matrix from scipy.sparse


In [9]:
# Import csr_matrix from sciy.sparse to view sparse matrix
from scipy.sparse import csr_matrix
mat_movies_users = csr_matrix(movies_users.values)
mat_movies_users

<9724x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

# Step 4 : Import nearestneighbours from sklearn.neighbors 

In [10]:
# Import NearestNeighbors from sklearn.neighbors to view cosine values nearest neighbours
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors = 20)
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

# Step 5 : Import process from fuzzywuzzy library

In [11]:
from fuzzywuzzy import process

# Step 6 : Create user define function   

In [12]:
#  Define a function and create a loop 
def recommender(movie_name,data, n):
    idices = process.extractOne(movie_name,movies['title'])[2]
    print('Movie Selected : ', movies['title'][idices], 'Index: ', idices)
    distance,indices = model_knn.kneighbors(data[idices], n_neighbors=n)
    for i in indices :
          print(movies['title'][i].where(i!=idices))

# Step 7 : Call function to check output

In [13]:
# call function to see the output
recommender("Toy Story",mat_movies_users,5)

Movie Selected :  Toy Story  Index:  0
0                                       NaN
2353                         'night Mother 
418                          Jurassic Park 
615          Independence Day (a.k.a. ID4) 
224     Star Wars: Episode IV - A New Hope 
Name: title, dtype: object


In [14]:
# Again call function   and check 10 users
recommender("Jumanji",mat_movies_users, 10)

Movie Selected :  Jumanji  Index:  1
1                                   NaN
322                     Lion King, The 
436                     Mrs. Doubtfire 
325                          Mask, The 
418                      Jurassic Park 
504                         Home Alone 
483    Nightmare Before Christmas, The 
506                            Aladdin 
512               Beauty and the Beast 
18      Ace Ventura: When Nature Calls 
Name: title, dtype: object


# End of the Program 